In [2]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy
import random
from gym import wrappers
import torch.optim as optim
import math
from torch.autograd import Variable

In [3]:
Env_Name='CartPole-v0'

Batch_Size=3
Replay_Memory_Size=10000
Target_Network_Update_Frequency=10000
Discount_Factor=0.99
Learning_Rate=0.00025
Initial_Exploration=1
Final_Exploration=0.1
Exploration_Decay=200
Episodes=500
Hidden_Layer_Size=256


device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device=",device)

env=gym.make(Env_Name)
#env=wrappers.Monitor(env,'./tmp/cartpole-v1-1',force=True)
#observation=env.reset()

Input_Shape=env.observation_space.shape[0]
Action_Shape=env.action_space.n
#print('Input Shape=',Input_Shape)
#print('Action Shape=',Action_Shape)


device= cuda


In [4]:

class ReplayMemory:
    def __init__(self,capacity):
        self.capacity=capacity
        self.memory=[]
    def push(self,transition):
        self.memory.append(transition)
        if len(self.memory)>self.capacity:
            del self.memory[0]
    def sample(self,batch_size):
        return random.sample(self.memory,batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:

class DQN(nn.Module):
    def __init__(self,input_shape=4,hidden_layer_shape=Hidden_Layer_Size,action_shape=2):
        super(DQN,self).__init__()
        self.l1=nn.Linear(input_shape,hidden_layer_shape)
        self.l2=nn.Linear(hidden_layer_shape,action_shape)
    def forward(self,x):
        x=F.relu(self.l1(x))
        x=self.l2(x)
        return x


In [6]:

model=DQN(input_shape=Input_Shape,hidden_layer_shape=Hidden_Layer_Size,action_shape=Action_Shape).to(device)


memory=ReplayMemory(Replay_Memory_Size)
optimizer=optim.Adam(model.parameters(),Learning_Rate)
steps_done=0
episode_durations=[]


In [7]:
def select_action(state):
    global steps_done
    sample=random.random()
    eps_threshold=Final_Exploration+(Initial_Exploration-Final_Exploration)*math.exp(-1.*steps_done/
                                                                                     Exploration_Decay)
    steps_done+=1
    if sample>eps_threshold:
        return  model(torch.tensor(state,dtype=torch.float32).to(device)).detach().max(1)[1].view(1,1)
    else:
        return torch.tensor([[random.randrange(2)]]).to(device)

In [53]:

def run_episode(e,env):
    state=env.reset()
    steps=0
    while True:
        env.render()
        action=select_action(torch.tensor([state],dtype=torch.float32))
        next_state,reward,done,_=env.step(action[0,0].item())

        if done:
            reward=-1

        memory.push((torch.tensor([state],dtype=torch.float32).to(device),
                    torch.tensor(action,dtype=torch.float32).to(device),
                    torch.tensor([next_state],dtype=torch.float32).to(device),
                    torch.tensor([reward],dtype=torch.float32).to(device)))
        #learn()
        state=next_state
        steps+=1
        if done:
            print("{2} Episode {0} finished after {1} steps".format(e,steps,'\033[92m' if steps>=195
                                                                    else '\033[99m'))
            episode_durations.append(steps)
           # plot_duration()
            break


In [56]:
Replay_Memory_Size
Batch_Size
memory.memory=[]

In [57]:
for e in range(1):
    run_episode(e,env)

/home/dell/myProjects/dqn_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/dell/myProjects/dqn_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


 Episode 0 finished after 28 steps


memory.memory


In [58]:
memory.memory

[(tensor([[ 0.0064, -0.0092, -0.0493,  0.0203]], device='cuda:0'),
  tensor([[1.]], device='cuda:0'),
  tensor([[ 0.0062,  0.1866, -0.0489, -0.2875]], device='cuda:0'),
  tensor([1.], device='cuda:0')),
 (tensor([[ 0.0062,  0.1866, -0.0489, -0.2875]], device='cuda:0'),
  tensor([[0.]], device='cuda:0'),
  tensor([[ 0.0100, -0.0078, -0.0546, -0.0106]], device='cuda:0'),
  tensor([1.], device='cuda:0')),
 (tensor([[ 0.0100, -0.0078, -0.0546, -0.0106]], device='cuda:0'),
  tensor([[0.]], device='cuda:0'),
  tensor([[ 0.0098, -0.2021, -0.0549,  0.2643]], device='cuda:0'),
  tensor([1.], device='cuda:0')),
 (tensor([[ 0.0098, -0.2021, -0.0549,  0.2643]], device='cuda:0'),
  tensor([[1.]], device='cuda:0'),
  tensor([[ 0.0058, -0.0062, -0.0496, -0.0452]], device='cuda:0'),
  tensor([1.], device='cuda:0')),
 (tensor([[ 0.0058, -0.0062, -0.0496, -0.0452]], device='cuda:0'),
  tensor([[1.]], device='cuda:0'),
  tensor([[ 0.0056,  0.1896, -0.0505, -0.3531]], device='cuda:0'),
  tensor([1.], devi

In [59]:
transitions=memory.sample(Batch_Size)
transitions

[(tensor([[ 0.0062,  0.1866, -0.0489, -0.2875]], device='cuda:0'),
  tensor([[0.]], device='cuda:0'),
  tensor([[ 0.0100, -0.0078, -0.0546, -0.0106]], device='cuda:0'),
  tensor([1.], device='cuda:0')),
 (tensor([[-0.0092,  0.5956, -0.1177, -1.2940]], device='cuda:0'),
  tensor([[1.]], device='cuda:0'),
  tensor([[ 0.0027,  0.7920, -0.1435, -1.6211]], device='cuda:0'),
  tensor([1.], device='cuda:0')),
 (tensor([[-0.0065, -0.3880, -0.0569,  0.3533]], device='cuda:0'),
  tensor([[0.]], device='cuda:0'),
  tensor([[-0.0142, -0.5822, -0.0499,  0.6275]], device='cuda:0'),
  tensor([1.], device='cuda:0'))]

In [60]:
batch_state,batch_action,batch_next_state,batch_reward=zip(*transitions)
batch_state,batch_action,batch_next_state,batch_reward

((tensor([[ 0.0062,  0.1866, -0.0489, -0.2875]], device='cuda:0'),
  tensor([[-0.0092,  0.5956, -0.1177, -1.2940]], device='cuda:0'),
  tensor([[-0.0065, -0.3880, -0.0569,  0.3533]], device='cuda:0')),
 (tensor([[0.]], device='cuda:0'),
  tensor([[1.]], device='cuda:0'),
  tensor([[0.]], device='cuda:0')),
 (tensor([[ 0.0100, -0.0078, -0.0546, -0.0106]], device='cuda:0'),
  tensor([[ 0.0027,  0.7920, -0.1435, -1.6211]], device='cuda:0'),
  tensor([[-0.0142, -0.5822, -0.0499,  0.6275]], device='cuda:0')),
 (tensor([1.], device='cuda:0'),
  tensor([1.], device='cuda:0'),
  tensor([1.], device='cuda:0')))

In [47]:
batch_reward

(tensor([1.], device='cuda:0'),
 tensor([1.], device='cuda:0'),
 tensor([1.], device='cuda:0'))

In [48]:
torch.cat(batch_reward)

tensor([1., 1., 1.], device='cuda:0')

In [49]:
Variable(torch.cat(batch_reward))

tensor([1., 1., 1.], device='cuda:0')

In [50]:
torch.tensor(torch.cat(batch_reward))

/home/dell/myProjects/dqn_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor([1., 1., 1.], device='cuda:0')

In [61]:
batch_state=torch.tensor(torch.cat(batch_state))
batch_action=torch.tensor(torch.cat((batch_action)))
batch_next_state=torch.tensor(torch.cat((batch_next_state)))
batch_reward=torch.tensor(torch.cat((batch_reward)))
print("batch_state=",batch_state)
print("batch_action=",batch_action)
print("batch_next_state=",batch_next_state)
print("batch_reward=",batch_reward)

batch_state= tensor([[ 0.0062,  0.1866, -0.0489, -0.2875],
        [-0.0092,  0.5956, -0.1177, -1.2940],
        [-0.0065, -0.3880, -0.0569,  0.3533]], device='cuda:0')
batch_action= tensor([[0.],
        [1.],
        [0.]], device='cuda:0')
batch_next_state= tensor([[ 0.0100, -0.0078, -0.0546, -0.0106],
        [ 0.0027,  0.7920, -0.1435, -1.6211],
        [-0.0142, -0.5822, -0.0499,  0.6275]], device='cuda:0')
batch_reward= tensor([1., 1., 1.], device='cuda:0')


/home/dell/myProjects/dqn_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/home/dell/myProjects/dqn_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/dell/myProjects/dqn_cartpole/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dell/my

In [64]:
current_q_values=model(batch_state)
print("batch_action=",batch_action)
print("q=",current_q_values.gather(1,batch_action))
print("current_q_values=",current_q_values)

batch_action= tensor([[0.],
        [1.],
        [0.]], device='cuda:0')


RuntimeError: gather_out_cuda(): Expected dtype int64 for index